In [2]:
import json
from web3 import Web3
from web3.middleware import geth_poa_middleware
import pickle
import requests
import pandas as pd
from datetime import datetime
import numpy as np
import math

In [3]:
web3 = Web3(Web3.HTTPProvider("https://bsc-dataseed.binance.org/"))
web3.middleware_onion.inject(geth_poa_middleware, layer=0)
web3.isConnected()

True

In [46]:
#Address = token contract
apikey = 'GKDPH6RRX5YSH1TYPMNHNVKGMB6I9V3I8T'
address = "0x5f4Bde007Dc06b867f86EBFE4802e34A1fFEEd63"

#Methods
approval_method = '0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925'
transfer_method = '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'
stake_method = ''


#Set block
currentblock = web3.eth.blockNumber
endblock = currentblock

#Set topic0
topic0 = approval_method

#Current block
currentblock = web3.eth.blockNumber
endblock = currentblock


#Otros contratos
claim_contract = ""
stake_contract = ""


#Liquidity Pool
pcs = "0x000000000000000000000000e98ac95A1dB2fCaaa9c7D4ba7ecfCE4877ca2bEa"
pcs_address = "0xe98ac95A1dB2fCaaa9c7D4ba7ecfCE4877ca2bEa"

pcs2 = "0x00000000000000000000000004DbC0634410B2486638C4A4BD5874f78d0E94BB"
pcs_address2 = "0x04DbC0634410B2486638C4A4BD5874f78d0E94BB"

#Set block
currentblock = web3.eth.blockNumber
endblock = currentblock

#Set topic0
topic0 = approval_method

#Wallets to check
check_wallets = [address, claim_contract, stake_contract, pcs_address, pcs_address2]

In [5]:
#Get ABI

url = "https://api.bscscan.com/api"
params = {
    "module": "contract",
    "action": "getabi",
    "address": address,
    "apikey": apikey
}

response = requests.get(url, params=params)

if response.status_code == 200:
    abi = response.json()["result"]
    print("Abi recibida correctamente")
else:
    print("Error en la solicitud")

Abi recibida correctamente


In [6]:
w3address = Web3.toChecksumAddress(address.lower())
contract = web3.eth.contract(address=w3address, abi=abi)

token = contract.functions.name().call()
symbol = contract.functions.symbol().call()

print(f"Abi del token: {token}, con simbolo: {symbol}, recibida correctamente")

Abi del token: Highstreet Token, con simbolo: HIGH, recibida correctamente


In [9]:
#Get "Internal Transactions" by Block Range
def get_internal_transactions(endblock, numberofblocks):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "account",
        "action": "txlistinternal",
        "startblock": endblock - numberofblocks,
        "endblock": endblock,
        "page": 1,
        "offset": 10,
        "sort": "asc",
        "apikey": apikey
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        transactions = response.json()["result"]
        return transactions
    else:
        return "Error en la solicitud"

#Get Logs by Address sin topic0
def get_logs_loop(address, endblock, blocks):
    if blocks % 1000 != 0:
        raise ValueError("El número de bloques debe ser múltiplo de 1000")
    total_logs = []
    for loop in range(int(blocks/1000)):
        logs = get_logs1000(address,endblock-loop*1000)
        total_logs = total_logs + logs
    return total_logs 


def get_logs1000(address, endblock):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": endblock - 1000,
        "toBlock": endblock,
        "address": address,
        "apikey": apikey
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        logs = response.json()["result"]
        return logs
    else:
        return "Error en la solicitud"

#Get Logs by Address
def get_logstopic_loop(address, endblock, blocks, topic0):
    if blocks % 1000 != 0:
        raise ValueError("El número de bloques debe ser múltiplo de 1000")
    total_logs = []
    for loop in range(int(blocks/1000)):
        logs = get_logs1000(address,endblock-loop*1000)
        total_logs = total_logs + logs
    return total_logs 


def get_logs1000topic(address, endblock, topic0):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": endblock - 1000,
        "toBlock": endblock,
        "topic0": topic0,
        "address": address,
        "apikey": apikey
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        logs = response.json()["result"]
        return logs
    else:
        return "Error en la solicitud"


#Get Logs by Address and From
def get_logstopic0_1_loop(address, endblock, blocks, topic0, topic1):
    if blocks % 1000 != 0:
        raise ValueError("El número de bloques debe ser múltiplo de 1000")
    total_logs = []
    for loop in range(int(blocks/1000)):
        logs = get_logs1000(address,endblock-loop*1000)
        total_logs = total_logs + logs
    return total_logs 


def get_logs1000topic0_1(address, endblock, topic0, topic1):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": endblock - 1000,
        "toBlock": endblock,
        "topic0": topic0,
        "topic0_1": "and",
        "topic1": topic1,                
        "address": address,
        "apikey": apikey
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        logs = response.json()["result"]
        return logs
    else:
        return "Error en la solicitud"

#Get a list of 'BEP-20 Token Transfer Events' by wallet
def get_token_transfers(wallet, address):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "account",
        "action": "tokentx",
        "contractaddress": address,
        "address": wallet,
        "page": 1,
        "offset": 1000,
        "startblock": 0,
        "endblock": 999999999,
        "sort": "asc",
        "apikey": apikey,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        transactions = response.json()["result"]
        return transactions
    else:
        return "Error in the request"

#Remove known wallets
def delete_wallet_from_df(df, check_wallets):
    for wallet_address in check_wallets:
        result = df['wallets'].isin([wallet_address.lower()]).any()
        print(f"Esta {wallet_address} en las wallets? {result}")
        if result:
            df = df.query('wallets != "{}"'.format(wallet_address.lower()))
    return df

In [10]:
## Extractor de Events/topics0
total_logs = get_logs_loop(address,endblock,100000)


#Save hash in a list
total_hash = []
for i in range(len(total_logs)):
    event_hash = total_logs[i]['topics'][0]
    total_hash.append(event_hash)

unique_event_hash = list(set(total_hash))


#Cantidad de logs extraidos
print(f"Se han obtenido {len(total_logs)} logs, en los cuales hay {len(unique_event_hash)} event hash unicos.")
print(unique_event_hash)


#Guarda los nuevos event hash
logs_unique = []
for value in unique_event_hash:
    if value not in logs_unique:
        logs_unique.append(value)

print(f"Actualmente hay almacenados {len(logs_unique)} event hash unicos que utilizaremos para reconocer los diferentes methods de topic0")


Se han obtenido 17488 logs, en los cuales hay 2 event hash unicos.
['0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef', '0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925']
Actualmente hay almacenados 2 event hash unicos que utilizaremos para reconocer los diferentes methods de topic0


In [17]:
#Un dia son aprox 28200 bloques, queremos calcular el finde pasado -5/-7 dias
currentblock = web3.eth.blockNumber
endblock = currentblock - (28200*4)
blocks = 28200 * 2.5
blocks = math.ceil(blocks / 1000.0) * 1000 #Multiplos de 1000

#Get data
logs_weekend = get_logs_loop(address, endblock, blocks)
token_tx = pd.DataFrame(logs_weekend)
token_tx.head()

,address,topics,data,blockNumber,blockHash,timeStamp,gasPrice,gasUsed,logIndex,transactionHash,transactionIndex
0,0x5f4bde007dc06b867f86ebfe4802e34a1ffeed63,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,0x00000000000000000000000000000000000000000000...,0x1813a54,0x5db884c4e21808dd9aee2c0a5c519f3008b35383fd4f...,0x63d84ae2,0x12a05f201,0x12cb0,0x3e,0xddac574e1caa1cfedc35e0aaf2a05c8fbf8ba439c36b...,0xd
1,0x5f4bde007dc06b867f86ebfe4802e34a1ffeed63,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,0x00000000000000000000000000000000000000000000...,0x1813a55,0x2253038fcca0721872f2332ecf57d8d4708320fcd5aa...,0x63d84ae5,0x12a05f200,0xcae8,0x32,0xf01e635d232e6f419034c371a6e62ac63765d0676dae...,0x11
2,0x5f4bde007dc06b867f86ebfe4802e34a1ffeed63,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,0x00000000000000000000000000000000000000000000...,0x1813a62,0xb3813462a339c6bcf8c30b3073bc5452bed95eb1d771...,0x63d84b14,0x12a05f201,0x12ee4,0x39,0x22f5f7b153af57836bf56423021e1e67ac1b4dd3290d...,0x6
3,0x5f4bde007dc06b867f86ebfe4802e34a1ffeed63,[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c...,0x00000000000000000000000000000000000000000000...,0x1813a69,0x3fa4d52d93c77e105f40020c6531abe7d9cd3ccfe64c...,0x63d84b31,0x12a05f201,0x12cb0,0x4a,0x74fccbb8dc21194dcec0ff620cc23bde48f1797520c6...,0x17
4,0x5f4bde007dc06b867f86ebfe4802e34a1ffeed63,[0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b...,0xffffffffffffffffffffffffffffffffffffffffffff...,0x1813a73,0x7cf3c96aa3858076e08516dfaab6e4af8922c92aca48...,0x63d84b51,0x12a05f200,0xb098,0xa7,0xcc13321ae9cd72673976a01bf788dea32fe0929e8f14...,0x49


In [49]:
# TOKEN LOG CLEANING

#Data Clean
# token_tx['timeStamp'] = token_tx['timeStamp'].apply(lambda x: int(x, 16))
# token_tx['timeStamp'] = token_tx['timeStamp'].apply(lambda x: datetime.fromtimestamp(x))

df_topics = pd.DataFrame(token_tx['topics'].tolist(), columns=['method', 'from', 'to'])
token_tx = pd.concat([token_tx, df_topics], axis=1)

token_tx = token_tx.drop(
    columns=[
        "topics",
        "address",
        "data",
        "blockNumber",
        "blockHash",
        "gasPrice",
        "gasUsed",
        "logIndex",
        "gasUsed",
        "transactionIndex",
    ],
    axis=1,
)

token_tx.sort_values(by='timeStamp', ascending=True, inplace=True)

#Filtrar la fecha
token_tx['intime'] = 0
for index, row in token_tx.iterrows():
    if row['timeStamp'].dayofyear < 31 and row['timeStamp'].dayofyear > 28 :
        token_tx.at[index, 'intime'] = 1

token_tx.head()

token_tx['intime'] = token_tx['intime'].astype(bool)
token_tx = token_tx[token_tx['intime'] == 1]

token_tx = token_tx.drop(columns=["intime"],axis=1,)

#Standarizar address, from & to
token_tx['to'] = "0x" + token_tx['to'].str[26:]
token_tx['from'] = "0x" + token_tx['from'].str[26:]

#Filtrar solo transferencias
token_tx = token_tx[token_tx['method'] == transfer_method]


In [51]:
token_tx.head()

,timeStamp,transactionHash,method,from,to
21578,2023-01-29 00:09:14,0xcb98d1707647e35f6494043b606dc6ed951db9c40df7...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0xe2fc31f816a9b94326492132018c3aecc4a93ae1,0x17d1027cd7671f60603796fb2cf755c9207c52cd
21579,2023-01-29 00:14:17,0x8b3c2ba731a5878048d06e18a2ba19a23295e669c7f3...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0x28cc814be3b994187b7f8bfed10516a84a671119,0x2887c6e3e5690485d75cb2d2afab17da9c7f7e99
21560,2023-01-29 00:29:08,0x50953f98a47d2ad3af4266b815dc3539f3dbbce95ab2...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0xe98ac95a1db2fcaaa9c7d4ba7ecfce4877ca2bea,0xfb2caa5f9660636dbc33bd5cc0b44f91d0962619
21564,2023-01-29 00:36:11,0xbea4c31c45bcbf81063e82b697c71c34d6dc359140f2...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0xdb6f1920a889355780af7570773609bd8cb1f498,0xe98ac95a1db2fcaaa9c7d4ba7ecfce4877ca2bea
21562,2023-01-29 00:36:11,0xbea4c31c45bcbf81063e82b697c71c34d6dc359140f2...,0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...,0xd3744b27813f2456ff3ddf78214bf57735d5f1a1,0xdb6f1920a889355780af7570773609bd8cb1f498


In [52]:
token_tx['from'].value_counts()

0xe98ac95a1db2fcaaa9c7d4ba7ecfce4877ca2bea    7728
0x39dd07ac211c33e796e6d89c4df9481e0b60ecfe    2095
0xd03ccfe61e66112472541888485ae67425973b01    1532
0xcd254486fb8f5a776d8780504b87a55b80c669dc    1205
0xf959cc2dffa1a6328a16074dbe98f5ed1535ff9e     565
                                              ... 
0xf325473e87b97401de52a2600aeda91952178d73       1
0x514e74747eeb8c79079715705a5c31a8ec3614cd       1
0x003f9cac18e2aa324bb489bd1f51f588690a8726       1
0xd4a9ff0d5865c041d1a056d12dbebb9ebebb7c48       1
0xc62f44943ceb6bc1cfc6811a2124ec6f4e490c49       1
Name: from, Length: 1506, dtype: int64

In [53]:
wallets = token_tx['from'].unique()
wallets = pd.DataFrame(wallets, columns=['wallets'])
wallets

,wallets
0,0xe2fc31f816a9b94326492132018c3aecc4a93ae1
1,0x28cc814be3b994187b7f8bfed10516a84a671119
2,0xe98ac95a1db2fcaaa9c7d4ba7ecfce4877ca2bea
3,0xdb6f1920a889355780af7570773609bd8cb1f498
4,0xd3744b27813f2456ff3ddf78214bf57735d5f1a1
...,...
1501,0xe8e207b7254c3d502259f003375f16e51501bec4
1502,0x535daa8e7a56577f1fab1ae4f7631054c584ed45
1503,0x730b428580d5131aa05018ebd88133cb37e51d18
1504,0x10a4751849d73cc931ad5583dec296d286cc1ad2


In [56]:
wallets = delete_wallet_from_df(wallets, check_wallets)

Esta 0x5f4Bde007Dc06b867f86EBFE4802e34A1fFEEd63 en las wallets? False
Esta  en las wallets? False
Esta  en las wallets? False
Esta 0x72121d60b0e2F01c0FB7FE32cA24021b42165A40 en las wallets? False
Esta 0xcC3A3Bc1d76Df321f94716E88224638C439267aa en las wallets? False
Esta 0xB4c88543Bd95617Fb61F3d031B8E8E19aF33201E en las wallets? False


In [57]:
wallets = delete_wallet_from_df(wallets, check_wallets)

Esta 0x5f4Bde007Dc06b867f86EBFE4802e34A1fFEEd63 en las wallets? False
Esta  en las wallets? False
Esta  en las wallets? False
Esta 0x72121d60b0e2F01c0FB7FE32cA24021b42165A40 en las wallets? False
Esta 0xcC3A3Bc1d76Df321f94716E88224638C439267aa en las wallets? False
Esta 0xB4c88543Bd95617Fb61F3d031B8E8E19aF33201E en las wallets? False


In [58]:
wallets.to_csv('high_bought_weekend.csv', index=False)